# Extending ImageJ: Ops

This notebook illustrates how to create new `Op` plugins, and run them with ImageJ's `OpService`.

The plugins are in Groovy cells, but coded in Java style.

In [1]:
%classpath config resolver scijava.public https://maven.scijava.org/content/groups/public
%classpath add mvn net.imagej imagej 2.0.0-rc-71
ij = new net.imagej.ImageJ()
"ImageJ v${ij.getVersion()} is ready to go."

Added new repo: scijava.public


ImageJ v2.0.0-rc-71 is ready to go.

## `narf` - a very simple op

Here is a "Hello, world" style op, to illustrate the basics:

In [2]:
import net.imagej.ops.AbstractOp
import net.imagej.ops.Op
import org.scijava.ItemIO
import org.scijava.plugin.Parameter
import org.scijava.plugin.Plugin
import org.scijava.plugin.PluginInfo

// The @Plugin annotation allows this Op to be discovered by the OpService.
// We declare the type of op, the name of the op, and any optional aliases...
@Plugin(type = Op.class, name = "narf")
public class Narf extends AbstractOp {
  
  // INPUTS, declared using @Parameter notation
  @Parameter
  private String input
  
  // OUTPUTS, declared using @Parameter notation
  @Parameter(type = ItemIO.OUTPUT)
  private String output
  
  @Override
  public void run() {
    // The job of the run method is to populate any outputs using the inputs
    output = "Egads! " + input.toUpperCase()
  }
}

// The @Plugin annotation is processed by the javac compiler,
// which is used to generate the metadata in class bytecode.
// Unfortunately, the Groovy compiler doesn't invoke the javac
// compiler, so we need to register the plugin manually!
narfInfo = new PluginInfo(Narf.class, Op.class)
ij.plugin().addPlugin(narfInfo)
narfInfo

name='narf', priority=0.0, enabled=true, pluginType=Op

Now you can start using your new Op.

In [3]:
// Execute our Op and get the result
result = ij.op().run("narf", "Put some trousers on")

Egads! PUT SOME TROUSERS ON

## `ramp` - an op which produces an image

This example illustrates how to create a new Op that constructs a ramp image.

In [4]:
import net.imagej.ops.AbstractOp
import net.imagej.ops.Op
import org.scijava.ItemIO
import org.scijava.plugin.Parameter
import org.scijava.plugin.Plugin
import org.scijava.plugin.PluginInfo
import net.imglib2.Cursor;
import net.imglib2.img.array.ArrayImg
import net.imglib2.img.array.ArrayImgs
import net.imglib2.img.basictypeaccess.array.DoubleArray
import net.imglib2.type.numeric.RealType
import net.imglib2.type.numeric.real.DoubleType

// The @Plugin annotation allows this Op to be discovered by the OpService.
// We declare the type of op, the name of the op, and any optional aliases...
@Plugin(type = Op.class, name = "ramp")
public class Ramp<T extends RealType<T>> extends AbstractOp {

  // INPUTS, declared using @Parameter annotation
  @Parameter(required = false)
  private int size = 256

  // OUTPUTS, declared using @Parameter notation
  @Parameter(type = ItemIO.OUTPUT)
  private ArrayImg<DoubleType, DoubleArray> rampImg

  @Override
  public void run() {
    rampImg = ArrayImgs.doubles(size, size)

    Cursor<DoubleType> c = rampImg.localizingCursor()
    long[] pos = new long[rampImg.numDimensions()]
    
    // Iterate the image and get the each pixel location
    // Every pixel value is assigned its locations sum,
    // so generate the ramp pattern image.
    while (c.hasNext()) {
      c.fwd()
      c.localize(pos)
      c.get().setReal(sum(pos))
    }
  }
  
  // a sum method to be called in our Op
  private float sum(long[] pos) {
    float sum = 0
    for (long p : pos) {
      sum += p
    }
    return sum
  }
}

// The @Plugin annotation is processed by the javac compiler,
// which is used to generate the metadata in class bytecode.
// Unfortunately, the Groovy compiler doesn't invoke the javac
// compiler, so we need to register the plugin manually!
rampInfo = new PluginInfo(Ramp.class, Op.class)
ij.plugin().addPlugin(rampInfo)
rampInfo

name='ramp', priority=0.0, enabled=true, pluginType=Op

In [5]:
ramp = ij.op().run("ramp")

<img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAQAAAAEACAYAAABccqhmAAAMJklEQVR42u3UBxcQAhiF4Xa0aWtHNLXQ3gOFUmkgDU2jPbV3RqXQkKJoWKlIUbRDe5GG9t5TO073+xn3vefcn/C8CRIkSPDf3SdMmFBPlCiRnjhxYj1JkiR60qRJ9WTJkunJkyfX77nnHv3ee+/VU6RIoadMmVJPlSqVnjp1aj1NmjR62rRp9XTp0un33Xeffv/99+vp06fXM2TIoGfMmFHPlCmTnjlzZj1Llix61qxZ9QceeEDPli2bnj17dj1Hjhx6zpw59Vy5cum5c+fW8+TJo+fNm1d/8MEH9YceekjPly+f/vDDD+uPPPKInj9/fr1AgQJ6wYIF9UKFCumFCxfWixQpoj/66KN60aJF9WLFiunFixfXS5QooZcsWVJ/7LHH9Mcff1x/4okn9FKlSumlS5fWy5Qpo5ctW1YvV66cXr58eb1ChQp6xYoV9UqVKumVK1fWq1SpoletWlWvVq2aXr16db1GjRp6zZo19SeffFJ/6qmn9KefflqvVauWXrt2bf2ZZ57Rn332Wf25557T69Spo9etW1d//vnn9Xr16un169fXGzRooL/wwgt6w4YN9UaNGumNGzfWmzRpor/44ov6Sy+9pL/88st606ZN9VdeeUVv1qyZ3rx5c71FixZ6y5Yt9VdffVVv1aqV3rp1a71NmzZ627Zt9Xbt2unt27fXX3vtNf3111/X33jjDf3NN9/UO3TooHfs2FHv1KmT3rlzZ71Lly56165d9W7duundu3fXe/Tooffs2VPv1auX3rt3b/2tt97S+/Tpo/ft21fv16+f3r9/f33AgAH/DRw48K598IMf/I74FQDwgx/8nvgHDRoUAQA/+MHvh18BAD/4we+Jf/DgwREA8IMf/H74FQDwgx/8nviHDBkSAQA/+MHvh18BAD/4we+Jf+jQoREA8IMf/H74FQDwgx/8nviHDRsWAQA/+MHvh18BAD/4we+Jf/jw4REA8IMf/H74FQDwgx/8nvhHjBgRAQA/+MHvh18BAD/4we+Jf+TIkREA8IMf/H74FQDwgx/8nvjffvvtCAD4wQ9+P/wKAPjBD35P/O+8804EAPzgB78ffgUA/OAHvyf+d999NwIAfvCD3w+/AgB+8IPfE/97770XAQA/+MHvh18BAD/4we+Jf9SoUREA8IMf/H74FQDwgx/8nvhHjx4dAQA/+MHvh18BAD/4we+Jf8yYMREA8IMf/H74FQDwgx/8nvjff//9CAD4wQ9+P/wKAPjBD35P/GPHjo0AgB/84PfDrwCAH/zg98Q/bty4CAD4wQ9+P/wKAPjBD35P/B988EEEAPzgB78ffgUA/OAHvyf+Dz/8MAIAfvCD3w+/AgB+8IPfE/9HH30UAQA/+MHvh18BAD/4we+Jf/z48REA8IMf/H74FQDwgx/8nvgnTJgQAQA/+MHvh18BAD/4we+Jf+LEiREA8IMf/H74FQDwgx/8nvgnTZoUAQA/+MHvh18BAD/4we+J/+OPP44AgB/84PfDrwCAH/zg98Q/efLkCAD4wQ9+P/wKAPjBD35P/J988kkEAPzgB78ffgUA/OAHvyf+KVOmRADAD37w++FXAMAPfvB74p86dWoEAPzgB78ffgUA/OAHvyf+Tz/9NAIAfvCD3w+/AgB+8IPfE/9nn30WAQA/+MHvh18BAD/4we+Jf9q0aREA8IMf/H74FQDwgx/8nvinT58eAQA/+MHvh18BAD/4we+J//PPP48AgB/84PfDrwCAH/zg98T/xRdfRADAD37w++FXAMAPfvB74p8xY0YEAPzgB78ffgUA/OAHvyf+mTNnRgDAD37w++FXAMAPfvB74p81a1YEAPzgB78ffgUA/OAHvyf+2bNnRwDAD37w++FXAMAPfvB74v/yyy8jAOAHP/j98CsA4Ac/+D3xf/XVVxEA8IMf/H74FQDwgx/8nvi//vrrCAD4wQ9+P/wKAPjBD35P/N98800EAPzgB78ffgUA/OAHvyf+b7/9NgIAfvCD3w+/AgB+8IPfE/+cOXMiAOAHP/j98CsA4Ac/+D3xf/fddxEA8IMf/H74FQDwgx/8nvjnzp0bAQA/+MHvh18BAD/4we+Jf968eREA8IMf/H74FQDwgx/8nvjnz58fAQA/+MHvh18BAD/4we+J//vvv48AgB/84PfDrwCAH/zg98T/ww8/RADAD37w++FXAMAPfvB74l+wYEEEAPzgB78ffgUA/OAHvyf+H3/8MQIAfvCD3w+/AgB+8IPfE//ChQsjAOAHP/j98CsA4Ac/+D3xL1q0KAIAfvCD3w+/AgB+8IPfE/9PP/0UAQA/+MHvh18BAD/4we+J/+eff44AgB/84PfDrwCAH/zg98S/ePHiCAD4wQ9+P/wKAPjBD35P/EuWLIkAgB/84PfDrwCAH/zg98T/yy+/RADAD37w++FXAMAPfvB74v/1118jAOAHP/j98CsA4Ac/+D3xL126NAIAfvCD3w+/AgB+8IPfE/+yZcsiAOAHP/j98CsA4Ac/+D3xL1++PAIAfvCD3w+/AgB+8IPfE/+KFSsiAOAHP/j98CsA4Ac/+D3xr1y5MgIAfvCD3w+/AgB+8IPfE/+qVasiAOAHP/j98CsA4Ac/+D3xr169OgIAfvCD3w+/AgB+8IPfE/+aNWsiAOAHP/j98CsA4Ac/+D3x//bbbxEA8IMf/H74FQDwgx/8nvh///33CAD4wQ9+P/wKAPjBD35P/H/88UcEAPzgB78ffgUA/OAHvyf+tWvXRgDAD37w++FXAMAPfvB74l+3bl0EAPzgB78ffgUA/OAHvyf+9evXRwDAD37w++FXAMAPfvB74t+wYUMEAPzgB78ffgUA/OAHvyf+jRs3RgDAD37w++FXAMAPfvB74t+0aVMEAPzgB78ffgUA/OAHvyf+zZs3RwDAD37w++FXAMAPfvB74t+yZUsEAPzgB78ffgUA/OAHvyf+rVu3RgDAD37w++FXAMAPfvB74t+2bVsEAPzgB78ffgUA/OAHvyf+7du3RwDAD37w++FXAMAPfvB74v/zzz8jAOAHP/j98CsA4Ac/+D3x//XXXxEA8IMf/H74FQDwgx/8nvh37NgRAQA/+MHvh18BAD/4we+J/++//44AgB/84PfDrwCAH/zg98S/c+fOCAD4wQ9+P/wKAPjBD35P/Lt27YoAgB/84PfDrwCAH/zg98S/e/fuCAD4wQ9+P/wKAPjBD35P/Hv27IkAgB/84PfDrwCAH/zg98T/zz//RADAD37w++FXAMAPfvB74t+7d28EAPzgB78ffgUA/OAHvyf+ffv2RQDAD37w++FXAMAPfvB74t+/f38EAPzgB78ffgUA/OAHvyf+AwcORADAD37w++FXAMAPfvB74j948GAEAPzgB78ffgUA/OAHvyf+Q4cORQDAD37w++FXAMAPfvB74j98+HAEAPzgB78ffgUA/OAHvyf+I0eORADAD37w++FXAMAPfvB74j969GgEAPzgB78ffgUA/OAHvyf+Y8eORQDAD37w++FXAMAPfvB74j9+/HgEAPzgB78ffgUA/OAHvyf+EydORADAD37w++FXAMAPfvB74j958mQEAPzgB78ffgUA/OAHvyf+U6dORQDAD37w++FXAMAPfvB74j99+nQEAPzgB78ffgUA/OAHvyf+M2fORADAD37w++FXAMAPfvB74j979mwEAPzgB78ffgUA/OAHvyf+c+fORQDAD37w++FXAMAPfvB74j9//nwEAPzgB78ffgUA/OAHvyf+CxcuRADAD37w++FXAMAPfvB74r948WIEAPzgB78ffgUA/OAHvyf+S5cuRQDAD37w++FXAMAPfvB74r98+XIEAPzgB78ffgUA/OAHvyf+K1euRADAD37w++FXAMAPfvB74v/3338jAO

Note that the resulting image is 256x256, since we did not specify the `size` parameter, and it is annotated with `required = false`. We can optionally pass the size to get a differently sized gradient:

In [6]:
smallRamp = ij.op().run("ramp", 120)

<img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAHgAAAB4CAYAAAA5ZDbSAAAFcklEQVR42u3RhWoQCgAF0Gs3ztmKrVPR2YHdMp0xA2ubhd3YjV1TzG0GduM2nYndYjd2Yzd238f7jcs9v3AAgEmSJGHSpEmZLFkyJk+enClSpGDKlCmZKlUqpk6dmmnSpGHatGmZLl06pk+fnhkyZGDGjBkZEBDATJkyMTAwkJkzZ2aWLFmYNWtWZsuWjdmzZ2eOHDmYM2dO5sqVi7lz52aePHmYN29e5suXj/nz52eBAgVYsGBBFipUiIULF2aRIkUYFBTEokWLslixYixevDhLlCjBkiVLMjg4mKVKlWLp0qVZpkwZli1bluXKlWP58uVZoUIFVqxYkZUqVWLlypVZpUoVVq1aldWqVWP16tVZo0YN1qxZk7Vq1WLt2rVZp04d1q1bl/Xq1WP9+vXZoEEDNmzYkCEhIWzUqBEbN27M0NBQNmnShE2bNmWzZs3YvHlzhoWFsUWLFmzZsiVbtWrF1q1bs02bNmzbti3btWvH9u3bs0OHDgwPD2dERAQjIyPZsWNHdurUiZ07d2aXLl3YtWtXduvWjd27d2ePHj3Ys2dP9urVi71792afPn3Yt29f9uvXj/379+eAAQM4cOBADho0iIMHD+aQIUM4dOhQDhs2jMOHD+eIESM4cuRIjho1iqNHj+aYMWM4duxYjhs3juPHj+eECRMI5+rmTpw4kXCubu6kSZMI5+rmTp48mXCubu6UKVMI5+rmTp06lXCubu60adMI5+rmTp8+nXCubu6MGTMI5+rmzpw5k3Cubu6sWbMI5+rmRkVFEc7VzZ09ezbhXN3cOXPmEM7VzZ07dy7hXN3cefPmEc7VzZ0/fz7hXN3cBQsWEM7VzV24cCHhXN3cRYsWEc7VzY2Ojiacq5sbExNDOFc3NzY2lnCubu7ixYsJ5+rmLlmyhHCubu7SpUsJ5+rmLlu2jHCubu7y5csJ5+rmrlixgnCubu7KlSsJ5+rmrlq1inCubu7q1asJ5+rmrlmzhnCubu7atWsJ5+rmrlu3jnCubu769esJ5+rmbtiwgXCubu7GjRsJ5+rmbtq0iXCubu7mzZsJ5+rmbtmyhXCubm5cXBzhXN3c+Ph4wrm6uQkJCYRzdXO3bt1KOFc3d9u2bYRzdXMTExMJ5+rmbt++nXCubu6OHTsI5+rm7ty5k3Cubu6uXbsI5+rm7t69m3Cubu6ePXsI5+rm7t27l3Cubu6+ffsI5+rm7t+/n3Cubu6BAwcI5+rmHjx4kHCubu6hQ4cI5+rmHj58mHCubu6RI0cI5+rmHj16lHCubu6xY8cI5+rmHj9+nHCubu6JEycI5+rmnjx5knCubu6pU6cI5+rmnj59mnCubu6ZM2cI5+rmnj17lnCubu65c+cI5+rmnj9/nnCubu6FCxcI5+rmXrx4kXCubu6lS5cI5+rmXr58mXCubu6VK1cI5+rmXr16lXCubu61a9cI5+rmXr9+nXCubu6NGzcI5+rm3rx5k3Cubu6tW7cI5+rm3r59m3Cubu6dO3cI5+rm3r17l3Cubu69e/cI5+rm3r9/n3Cubu6DBw8I5+rmPnz4kHCubu6jR48I5+rmPn78mHCubu6TJ08I5+rmPn36lHCubu6zZ88I5+rmPn/+nHCubu6LFy8I5+rmvnz5knCubu6rV68I5+rmvn79mnCubu6bN28I5+rmvn37lnCubu67d+8I5+rmvn//nnCubu6HDx8I5+rmfvz4kXCubu6nT58I5+rmfv78mXCubu6XL18I5+rmfv36lXCubu63b98I5+rmfv/+nXCubu6PHz8I5+rm/vz5k3Cubu6vX78I5+rm/v79m3Cubu6fP38I5+rm/v37l3Cubu6/f/8I5+rm/u8/rOccSfpV0IIAAAAASUVORK5CYII=" />

## `blobs` - a more complex op

This next example illustrates how to create a new op that constructs a 'random blobs' image.

In [7]:
import java.util.Random
import net.imagej.ops.AbstractOp
import net.imagej.ops.Op
import net.imglib2.RandomAccess
import net.imglib2.RandomAccessibleInterval
import net.imglib2.img.array.ArrayImgs
import net.imglib2.type.numeric.RealType
import net.imglib2.util.IntervalIndexer
import net.imglib2.util.Intervals
import org.scijava.ItemIO
import org.scijava.log.LogService
import org.scijava.plugin.Parameter
import org.scijava.plugin.Plugin
import org.scijava.plugin.PluginInfo

// The @Plugin annotation allows this Op to be discovered by the OpService.
// We declare the type of op, the name of the op, and any optional aliases...
@Plugin(type = Op.class, name = "blobs")
public class RandomBlobs<T extends RealType<T>> extends AbstractOp {

  // OUTPUTS, declared using @Parameter notation
  @Parameter(type = ItemIO.OUTPUT)
  private RandomAccessibleInterval<T> image
  
  @Parameter
  private LogService log
  
  @Parameter
  private int blobNum
  
  @Parameter
  private int blobSize
  
  @Parameter
  private int xDim
  
  @Parameter
  private int yDim
  
  @Parameter(required = false)
  private long seed = 0xcafebabe
  
  @Override
  public void run() {
    // produce a XxY float64 array-backed image using the input parameters
    image = ArrayImgs.doubles(xDim, yDim)
    long[] pos = new long[image.numDimensions()]
    
    long[] blobCenter = new long[image.numDimensions()]
    long[] dims = new long[image.numDimensions()]
    image.dimensions(dims)
    
    // get the total number elements of the image
    long total = Intervals.numElements(image)
    
    Random r = new Random(seed)
    
    RandomAccess<T> ra = image.randomAccess(image)
    
    // Iterate to generate each blob
    for (int i = 0; i < blobNum; i++) {
      // generate a random positon in [0, total)
      long index = (long) (r.nextDouble() * total)
      // convert the linear index to the 2-D index
      // For example, index = 59662, dims = [256,256],
      // then blobCenter = [14,233]
      IntervalIndexer.indexToPosition(index, dims, blobCenter)
      
      // For generating current blob, it is necessary to scan
      // the whole image to determine the elements which are
      // locate in the radius of the blobCenter.
      for (int j = 0; j < total; j++) {
        IntervalIndexer.indexToPosition(j, dims, pos)
        double dist = distance(pos, blobCenter)
        if (dist > blobSize) {
          continue
        }
        
        // This element is in the radius of the blobCenter, so it is 
        // assigned with value inversely proportional to the distance.
        // Namely, if the distance is 0.0, then the norm is 1.0; if the
        // distance is blobSize, then the norm is 0.0, and so on.
        ra.setPosition(pos)
        double norm = 1.0 - dist / blobSize
        ra.get().setReal(Math.max(ra.get().getRealDouble(), norm))
      }
    }
  }
  /**
   * Computes distance between the given position and a center point.
   */
  private double distance(long[] pos, long[] center) {
    long sumDistSquared = 0
    for (int d = 0; d < center.length; d++) {
      long dist = pos[d] - center[d]
      sumDistSquared += dist * dist
    }
    return Math.sqrt(sumDistSquared)
  }
}

// The @Plugin annotation is processed by the javac compiler,
// which is used to generate the metadata in class bytecode.
// Unfortunately, the Groovy compiler doesn't invoke the javac
// compiler, so we need to register the plugin manually!
blobsInfo = new PluginInfo(RandomBlobs.class, Op.class)
ij.plugin().addPlugin(blobsInfo)
blobsInfo

name='blobs', priority=0.0, enabled=true, pluginType=Op

In [8]:
blobs = ij.op().run("blobs", 30, 15, 256, 256)

<img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAQAAAAEACAYAAABccqhmAAAOm0lEQVR42u2dz8uN3RfGn6QkESYKJYok5dfEhBJFSlFkZqAwwkBRBiYUExMZUMrEQKYGkpmhlLGBsZl/YX+7dr1vb988z3POufePtdf6vHV1Oufc9z7e+7nvtde+1nWtvbS0tJQA+Adr1qxJmzZtStu3b0979+5Nhw4dSsePH0+nTp1K586dSxcuXEiXLl1Kly9fTleuXMmveq/P9b2O0/E6T+drHI2ncbm+q2PdunVp27Ztaf/+/fk66ppevXo13bp1K92/fz89efIkvXjxIr19+zZ9+PAhffz4MX3+/Dl9+fIlv+q9Ptf3Ok7H6zydr3E0nsbV+PqdJS46+C82btyYH9p9+/alo0ePphMnTqSzZ8+mixcv5hvo2rVr6fr16+nGjRvp5s2b+VXv9bm+13E6XufpfI2j8TQu13d1bNiwIe3cuTMdPHgwX0MFVl3bO3fupEePHqXnz5+nN2/e5If806dP6evXr+nbt2/p+/fv+VXv9bm+13E6XufpfI2j8TSuxtfvEADAv1i/fn2eFTRz6+E9efJkOn/+fJ7pdfNoFtGNdO/evTyrPHjwIL/qvT7X9zpOx+s8na9xNJ7G1fhc55WhbGnXrl3p8OHDOZtSdqUAq2v8+PHj9PLly/Tu3bs80+th//HjR/r582f69etXftV7fa7vdZyO13k6X+NoPI2r8fU7BACQsXbt2rR169a0e/funL5rltBDrFldN87t27fzw67ZRDfU06dP07Nnz/Kr3utzfa/jdLzO0/kaR+NpXI2v3+F6L4/Nmzfna6XAefr06bzEUpala6tr/erVq/T+/fuc7mvG10P/+/fv9OfPn/yq9/pc3+s4Ha/zdL7G0XgaV+PrdwgA4N/Uc8eOHenAgQN5jag0XjO5Hua7d++mhw8f5vWkUkrNKrqxXr9+nV/1Xp/rex2n43Weztc4Gk/janz9Dtd7eWzZsiXt2bMnHTt2LJ05cyZfQy21lG0p4OqaK73Xml9pv2Z+Pfz6T696r8/1vY7T8TpP52scjadxNb5+hwAAMkGn2Vk3xJEjR3KKqLW80nnN6HqoNYuIVNK6UqmlZhfdYHrVe32u73Wcjtd5Ol/jaDyNq/H1OxCCZAAgMPOs3+O6wwGAoMwzywCqACDwrKPf47qjAwBB1536Pa67DT6GAACaM8/6vagPt2Z4pfnKghQIdS30qvf6/J/vG1ZkeADIAMgAamss9LBpxlXarbW3lkK6FgqIetV7fa7vdVxDTQYPABwAHEBNdaUeNs24eti19hYBp2uhgKisSK96r8/1vY7T8Y1UmTwAMM9UAWqk+/I/6GHTjKu0W2tvpeC6FgqIyoq0NNKr3utzfa/jdLzOa+DL4AEouX4bUeSCDsC+o7KiM5OHu/T6TcePpHlHCTiGo3JKJqF7U/+uv1x7HvIa6zedN5L7DS/AGI7KCvciD3qt9dsKURc3II7Kha9h4WyUB56OOPQDGC2LKshH8dDTEYf//8A8Csw6MyAZUOBKCsw6a2A4kMBaCph1WHCqIIHVlMwq1MFj6yCC+ylYV6KEi62EDO6ohFlGCw/IAAIz67jhABxAYGYdPzwowZFQBRiUWacjDihZJUEHMBizTgYAalS0QisBR2LW4QBArYrWAFoSlFFUAUDtipZhNSmsKDoA0KqiZdBPQl0UJSBoXdEy5KhEGYUXAPS6Pww4KskAcAOCnhliZ0clHAD9AIAFjqiTo5IqAB1xgKV7ubGjEh0AHXFAYJ0ISkA64oDASlG8AHTEiUXozdvXwLlXBDcgHXFiufbm/TuSAcCs0xFn4Gs6NZODA4BZBwNmVSW5HMdeEXoCAn+dhmvcc069InQFBv7+/jWyTqdeEXrNA18ZYE3eyaFXhF7zwA8H1KLy5MwrQq954KcK1EJ74swrwk0KfOhAWqpPHVW0uFGBDyVoS/+Jo4oWNyrw4QVp7dpzUtHiZgU+3KA9FHsOKlrcrMBHP4gemn0HFS3YeuDDQ9/TtTfwM0K9Hvjw0LPDU8EAgGIPjNYVmh2eCgQANPtg1AyAHZ4mBgBce2BkDoAdniYGAHz7YOQqADs8TQgA9MQHHrpCs8PTAgGAXXGAl67Q3MsLBACiJvDUFZpsdg6wbgIeu0LDZ80ImFPgsSs0Fa0ZQe0UeO0KjaZlBqCeAt67QqNqXQHop0GErtD4WpaB833PgEFOoOdsjLOVDAAYqA4wG8MBAJYFzMZUAQAA6AAAACgBAQB4AQAAuAEBAPQDAADQEQgAQE9AAECNAICDCoDgAQAHFUBxSABAsw3wHEQPAERQgOuQAEAqCOg7QAAgFQR0HiIAkAoCo0DHQgAgFQy85kfJSgAwmQrCPdRf4uFlIQCYSwXhHtoANysBwFwqCPfQLtujnwUBwEwqCPfQFnS0IgCYSQUpQ/X5m9PTkgBgIhWkDNUedLUmAJhIBSlD9QH7WhAAuqeClKH6gZ2tCADdU0HKUGQABICgNwJlKDgAAkDgVJAyFFUAAoABK2+vDIAbEB3A4FJvP1beHqkgKWjc8q8TqbcvK2/rmRgSKqYAzJHU25+Vt2UqSBkqlgTcodTbp5W3VSpIBhDHBOZU6u3XytsiFYQDiGMDdyr19h3FayvyqALEaATjWOrtex1X+w+FDsB/KzjnUm//TG7NVA0loP9msM6l3jFquTXJGrwAftvBBwjwcdRctco1uAHHVpGupNQLsMSLpeeuJdigHwBeg0FJ3niOrlqSTToC4TYcsMwb08pbw7RBT0D6DQwo9KKri/UyFA8i9+awAWAWAsZjlGVfADKA0BnAvCUYj+ssdgaCAwjJASw6+3llWtkbkCpAiCpAifUvclpAx6EBA0ApBhw5LaDj0IABoGQNfCQ5Lek9HYfCewFqqOCsy2kh+Og4hBuw8sWxKqelxEfHIfoBNEqPLMlpEfnQcYiOQB0IEgtyWmS+gJ6AnUokFmZajD7AaRY4hkii51obqy9wzAONI5PswbbT7AM4rwSNZZRoXW+n3RdwrgXBKhlYBQYAVsnAOnAAsEoGdoIBwAMS2AsOACly4G4wAECSBe4HBwBlMjIAQABAKAMHAAgASGWpAgACAGYZdACAABDbKokSEBAAgjfMwAsACACBW2bhBgQEgOBNM+kHAAgAMa2SdAQCBABAT0BAACAI0BUYEAAA+wIAAgABgAAACAAsAVgCAAIAJCAkICAA+AVlQEAACLzmRwgECAABgRQYEAACqwQxAwECQFCfAHZgQAAwWkNvkT3QEAQQAIzW0FtkD7QEAwQAgzX0Vgo8moKC8AGgRKpduobeSoFHW3AQNgCUIupq1NBbKfDYGASEDACliLpaNfRWCjwyABAqAJQm6mrV0Fsp8OAAQJgAUMPsUquG3kqBRxUAhAkAtcwuNWrorRR46ABAiABQ0+xSY/ZspcBDCQjco7bZpfT6ufXMixcAuEbtG7w0g9567Y0bELhG7RS3dA29B/tOPwDgFi1IrpIZQK/6Ox2BgEu0KHOV5AB6KfDoCQhcooXQpWRw6anAoyswcIcWUteSy4veCjw9tOwLANygldmlBMHYU4E3qzlqxN2PARlAdbNLiRJjLwVeiRmfdB+E5QA0fokaemsFHmt+QBWgUKpdoobeUoEH6w/QARRMtUvU0Fsq8Kj7A5SAhVPtqbNpKwUeyj9Qqz1eOC/Af1PtqevpFjMx2n9Qqz1eSDfg/9/wUxj1Fmtx3H+gRnu8sP0A/pbyLhJRW7Dx+P9BwApQP9Jr3jVV7WhMByAQsAI01v90zfUYPQAh4AJWgMZMe2rcEHQBhoALWAEKSXywDwAEHBWg5QKA+9LHMmAnoNgEXNAKUEDxAxkABBwVoNkCQCTAAfRHLwIucAWIm44qgJ01fy8CLvDfnhuPWcAG29+TgAuc/XHzsQ60kX31JOAC8z